In [ ]:
import mysql.connector
import sqlalchemy

from common.utils import Properties, Config
from modelandpredict.traincovid import traincovid, predictcovid
from modelandpredict.trainhousing import trainhousing, predicthousing

In [ ]:
credentials = Properties('../resources/credentials.properties').get_properties()
user = credentials['user']
pw = credentials['pw']
address = credentials['address']
db = credentials['db']
db_conn = mysql.connector.connect(
    host=address,
    user=user,
    password=pw,
    database=db,
)
engine = sqlalchemy.create_engine("mysql+pymysql://{user}:{pw}@{address}/{db}"
                                  .format(user=user,
                                          pw=pw,
                                          address=address,
                                          db=db))
config = Config.get_instance()
valid_cities = config['housing_valid_cities'].split('/')

In [ ]:
for city in valid_cities:
    traincovid(city, db_conn)

print("Complete train covid")

In [ ]:
for city in valid_cities:
    trainhousing(city, db_conn)

print("Complete train housing")

In [ ]:
sql = "DELETE FROM covid_prediction"
cursor = db_conn.cursor()
cursor.execute(sql)
db_conn.commit()
for city in valid_cities:
    predictcovid(city, db_conn, engine)

print("Complete predict covid")

In [ ]:
sql = "DELETE FROM housing_median_prediction"
cursor = db_conn.cursor()
cursor.execute(sql)
db_conn.commit()
for city in valid_cities:
    predicthousing(city, db_conn, engine)

In [ ]:
db_conn.close()